# Импорт библиотек и загрузка данных

In [103]:
import json
import cv2 
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import pandas as pd
from collections import Counter
from functools import lru_cache
import os
import shutil
import re
import logging
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch

## Считываем данные

In [30]:
class ImageDataset:
    def __init__(self, path, path2image, n_classes = 15):
        with open(path, 'r') as file:
            data = json.load(file)
        self.dict = data
        self.path2image = Path(path2image)
        self.N = n_classes
        
    def __len__(self):
        return len(self.dict)

    def _encode_label(self, classes):
        label = np.zeros(self.N)
        for c in np.array(classes, dtype = np.int16) - 1:
            label[c] = 1
        return label
        
    def sample(self, clss = 1, count = 15):
        files = []
        while len(files) < count:
            indx = np.random.randint(0, self.__len__())
            labels = self.dict[indx]["OUTPUT:classes"]
            try:
                if clss in self.parse(labels):
                    files.append(self.path2image / self.file_name(indx))
            except:
                continue
        return files
    
    
    def create_path(self, indx):
        return str(self.path2image / self.file_name(indx))
        
    def __getitem__(self, indx):
        img = cv2.imread(self.create_path(indx))
        clases = self.parse(self.dict[indx]["OUTPUT:classes"])
        label = self._encode_label(clases)
        sample = {'image': img,
                  'label': label}
        return sample
            
    def file_name(self, indx):
        return self.dict[indx]['file_name']
    
    def resize(self, img, scale=2):    
        shape = (np.array(img.shape[:2])/scale).astype(np.int32)
        return cv2.resize(img,shape)
    
    
    def parse(self, st):        
        return [int(x) for x in re.findall("\d+", st)]

<>:51: SyntaxWarning: invalid escape sequence '\d'
<>:51: SyntaxWarning: invalid escape sequence '\d'
C:\Users\maksi\AppData\Local\Temp\ipykernel_8896\3186870174.py:51: SyntaxWarning: invalid escape sequence '\d'
  return [int(x) for x in re.findall("\d+", st)]


In [31]:
class TorchImageDataset(ImageDataset, Dataset):
    def __init__(self, path, path2image, imgsz=256, MEAN = (0.485, 0.456, 0.406), STD = (0.229, 0.224, 0.225)):
        super().__init__(path, path2image)
        self.imgsz = imgsz
        self.fransform_img= transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((self.imgsz, self.imgsz)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),    
                    transforms.ColorJitter(brightness=0.2, 
                                                   contrast=0.2, 
                                                   saturation=0.2, 
                                                   hue=0.2),
                    transforms.Normalize(mean=MEAN, std=STD),
        ])
            
    @lru_cache(10000)
    def __getitem__(self,indx):
        img, label = super().__getitem__(indx).values()
        if img is not None:
            img = self.fransform_img(img)
            return {'image':img,'label':label}

## Подготовка файла

In [72]:
def merge_json_files(directory, output_file):
    merged_data = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'r') as file:
                data = json.load(file)
                
                for item in data:
                    if is_valid_item(item):
                        merged_data.append(item)

    seen = set()
    unique_data = []
    
    for item in merged_data:
        serialized_item = json.dumps(item, sort_keys=True)
        if serialized_item not in seen:
            seen.add(serialized_item)
            unique_data.append(item)

    with open(output_file, 'w') as outfile:
        json.dump(unique_data, outfile, indent=4)

def is_valid_item(item):
    if 'file_name' not in item or 'OUTPUT:classes' is None:
        print(f"Skipping invalid item: {item}")
        return False  

    classes = item.get('OUTPUT:classes', None)
    
    if classes is None:  # Пропускаем, если classes == None (или JSON null)
        print(f"Skipping item with invalid classes (None/null): {item}")
        return False 
    return True 

<>:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
C:\Users\maksi\AppData\Local\Temp\ipykernel_8896\1301915487.py:28: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if 'file_name' not in item or 'OUTPUT:classes' is None:


In [73]:
directory = './' 
output_file = 'merged.json' 

merge_json_files(directory, output_file)

Skipping item with invalid classes (None/null): {'task_id': '1df19709-dc7a-448b-a5f2-61a8f65d4853', 'task_type': 'PROD', 'file_name': '35TimePhoto_20241001_063231 (54).jpg', 'organization_id': 'e2f6ea8c-516a-4a43-aaa1-0544a9a60ec9', 'person_id': 'bc52b760-0a6a-47e1-bb86-c01f0e931d87', 'item_id': 'c8323992-6b92-43dd-8996-e199afff29a9', 'item_type': 'data', 'assignment_id': 'b4816cb6-1919-489a-b0d3-9682cdebd2ae', 'status': 'ACCEPTED', 'marker_id': '3a933646-2de5-486d-8dc0-dacea62c685a', 'start_date': '2024-11-13T20:17:42.410395+03:00', 'end_date': '2024-11-13T20:18:31.125022+03:00', 'control_passed': None, 'pool_ids': '["bb899827-7127-4aaf-81db-531ffdbddea6"]', 'price': 5.0, 'consistency': 0.0, 'quality': None, 'submitted_at': '2024-11-13T20:18:31.125022+03:00', 'skipped_at': None, 'expired_at': None, 'accepted_at': '2024-11-13T20:18:31.125022+03:00', 'rejected_at': None, 'started_at': '2024-11-13T20:17:42.410395+03:00', 'reviewer_id': None, 'reviewer_comment': None, 'OUTPUT:image': '/ap

## Получаем датасет

In [74]:
dataset = TorchImageDataset("./merged.json",  "./Resize/")

In [75]:
len(dataset)

37753

## DataLoader

In [76]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():  
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device       

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            img,label = tuple(b.values())
            img = to_device(img,self.device)
            label = to_device(label, self.device)
            yield {"image":img,"label":label}

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [77]:
def split_data(dataset, train_ratio=0.7, val_ratio=0.1):

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

    return train_data, val_data, test_data

In [97]:
train_data, val_data, test_data = split_data(dataset, train_ratio=0.2, val_ratio=0.025)

In [98]:
len(train_data)

7550

In [80]:
device = torch.device("cuda:0")

In [99]:
train_loader = DeviceDataLoader(DataLoader(train_data, batch_size=8,num_workers = 0), device=device)
val_loader   = DeviceDataLoader(DataLoader(val_data, batch_size=8,num_workers = 0), device=device)
test_loader   = DeviceDataLoader(DataLoader(test_data, batch_size=8,num_workers = 0), device=device)

# Обучение модели

In [100]:
class ModelMultilabel(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes        
        self.model = None
        self.model = self.__efficientnet_b5()
        
    def __efficientnet_b5(self):
        model = models.efficientnet_b5(pretrained=True)
        model.classifier[1] = nn.Linear(2048,self.n_classes)
        return model
    def forward(self, input):
        out = self.model(input)
        out = F.sigmoid(out)
        return out

def train_model(model, dataloader_train, dataloader_valid, learningRate, num_epochs, device = 'cuda:0'):
    logger = logging.getLogger('api_log')
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(f'log_{learningRate}.txt')
    file_handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.debug(f"Start_train  lr={learningRate}")
    
    # Установка оптимизатора с текущей скоростью обучения
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    criterion = nn.CrossEntropyLoss()
    # Цикл обучения
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []

        for s in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)  # Выходы модели
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())
        # Средняя потеря за эпоху
        epoch_loss = running_loss / len(dataloader_train)
        # precision = precision_score(all_labels, all_predictions, average='weighted')
        # recall = recall_score(all_labels, all_predictions, average='weighted')
        # accuracy = accuracy_score(all_labels, all_predictions)
        f_score = F_score(all_predictions, all_labels)
        # print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, "
        #       f"Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, F_score: {f_score:.4f}")
        
        # logger.debug(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f},  F_score: {f_score:.4f}")
        validate_model(model, dataloader_valid, logger)    
    # Сохранение модели
    torch.save(model.state_dict(), f'model_{learningRate}.pth')
    
def validate_model(model, dataloader_valid, logger, device='cuda:0'):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for s in tqdm(dataloader_valid, desc='Validation'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())

    # precision = precision_score(all_labels, all_predictions, average='weighted')
    # recall = recall_score(all_labels, all_predictions, average='weighted')
    # accuracy = accuracy_score(all_labels, all_predictions)
    f_score = F_score(all_predictions, all_labels)
    # print(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")
    # logger.debug(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
    logger.info(f"Validation Results -  F_score: {f_score:.4f}")


def F_score(output, label, threshold=0.5, beta=1): #Calculate the accuracy of the model
    if isinstance(output, list):
        output = np.array(output)
    if isinstance(label, list):
        label = np.array(label)
    print(label.shape, output.shape)
    assert (output.shape == label.shape),'shape is different'
    prob = output > threshold
    label = label > threshold

    TP = (prob == label).sum()
    TN = (np.bitwise_not(prob) == np.bitwise_not(label)).sum()
    FP = (prob == np.bitwise_not(label)).sum()
    FN = (np.bitwise_not(prob) == label).sum()
    print(TP, TN, FP, FN)
    precision = np.mean(TP / (TP + FP + 1e-12))
    recall = np.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2


In [101]:
model = ModelMultilabel(15).to(device)

In [102]:
train_model(model, train_loader, val_loader, 0.001, 30)

Training Epoch 1/30: 100%|██████████| 944/944 [17:46<00:00,  1.13s/it]


(7550, 15) (7550, 15)
94913 94913 18337 18337


Validation: 100%|██████████| 118/118 [02:13<00:00,  1.13s/it]


(943, 15) (943, 15)
12163 12163 1982 1982


Training Epoch 2/30: 100%|██████████| 944/944 [03:20<00:00,  4.71it/s]


...

(943, 15) (943, 15)
12704 12704 1441 1441


Training Epoch 30/30: 100%|██████████| 944/944 [03:16<00:00,  4.81it/s]


(7550, 15) (7550, 15)
104437 104437 8813 8813


Validation: 100%|██████████| 118/118 [00:06<00:00, 19.65it/s]


(943, 15) (943, 15)
12783 12783 1362 1362


In [106]:
torch.save(model, "model.pth")

In [ ]:
model = torch.load("model.pth")
model.eval()

## Метрики

In [108]:
def evaluate_f1_score(model, filename, device='cuda:0', threshold=0.5):
    test_dataset = TorchImageDataset(filename, 
                                    "./Resize/",
                                   256)
    valid_data = []
    for i in range(len(test_dataset)):
        # if i != 1752:
            valid_data.append(test_dataset[i] or "")
    dataloader = DeviceDataLoader(DataLoader(valid_data, batch_size=8,num_workers = 0), device=device)
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device)

            outputs = model(inputs)
            outputs = (outputs > threshold).float()

            all_predictions.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_predictions = np.vstack(all_predictions)
    all_labels = np.vstack(all_labels)
    
    f1 = f1_score(all_labels, all_predictions, average='weighted')  
    print(f"F1-Score: {f1:.4f}")
    return f1

In [109]:
evaluate_f1_score(model, "pool_8_1_1.json", device=device)

Evaluating: 100%|██████████| 283/283 [00:14<00:00, 19.32it/s]

F1-Score: 0.7546


np.float64(0.7545780617299203)

In [110]:
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        outputs = (outputs > 0.5).float()

        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
    
all_predictions = np.vstack(all_predictions)
all_labels = np.vstack(all_labels)
    
f1 = f1_score(all_labels, all_predictions, average='weighted')  
print(f"F1-Score: {f1:.4f}")

Evaluating: 100%|██████████| 3658/3658 [1:15:03<00:00,  1.23s/it]

F1-Score: 0.7568
